In [79]:
%%writefile app.py

import streamlit as st
import plotly.express as px
import pandas as pd
import numpy as np
from funpymodeling.exploratory import freq_tbl 
import matplotlib.pyplot as plt
import scipy.special as special
from scipy.optimize import curve_fit
import seaborn as sns
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.linear_model import LogisticRegression
import plotly.graph_objects as go


# Configuración de la página de Streamlit
st.set_page_config(page_title="Dashboard de Regresiones", page_icon="📊", layout="wide")

# Definimos la instancia de streamlit con cache
@st.cache_resource

# Creamos la función de carga de datos
def load_data():
    # Lectura del archivo csv con índice
    df = pd.read_csv("hongkong_clean.csv")

    # Selección de columnas numéricas del dataframe principal
    numeric_df = df.select_dtypes(['float', 'int'])
    numeric_cols = numeric_df.columns

    # Función para análisis univariado y filtrado
    def univariate_analysis(variable):
        table = freq_tbl(df[variable])  # Genera tabla de frecuencias
        filtro = table[table['frequency'] > 1]  # Filtro para valores con frecuencia mayor a 1
        filtro_index = filtro.set_index(variable)  # Ajusta el índice al valor de la variable
        numeric_df = filtro_index.select_dtypes(['float', 'int'])  # Selección de columnas numéricas
        return filtro_index, numeric_df.columns

    # Realizamos el análisis univariado para las variables específicas
    Filtro_index1, numeric_cols1 = univariate_analysis('host_is_superhost')
    Filtro_index2, numeric_cols2 = univariate_analysis('host_identity_verified')
    Filtro_index3, numeric_cols3 = univariate_analysis('neighbourhood_cleansed')
    Filtro_index4, numeric_cols4 = univariate_analysis('room_type')
    Filtro_index5, numeric_cols5 = univariate_analysis('instant_bookable')

    # Devolvemos los dataframes y las columnas numéricas resultantes
    return Filtro_index1, Filtro_index2, Filtro_index3, Filtro_index4, Filtro_index5, df, numeric_df, numeric_cols, numeric_cols2, numeric_cols3, numeric_cols4, numeric_cols5
    

# Cargar los datos obtenidos
Filtro_index1, Filtro_index2, Filtro_index3, Filtro_index4, Filtro_index5, df, numeric_df, numeric_cols, numeric_cols2, numeric_cols3, numeric_cols4, numeric_cols5 = load_data()

# SIDEBAR
st.sidebar.title("DASHBOARD // HONG KONG 🌏")
st.sidebar.header("Panel de selección")
st.sidebar.subheader("Selecciona el análisis que deseas realizar")

# FRAMES
Frames = st.selectbox(label="Análisis de correlaciones", options=['Inicio',"Regresion lineal simple", "Regresion lineal multiple", "Regresion no lineal", "Heatmap"])

if Frames == "Inicio":
    # Título de la página con estilo
    st.markdown("<h1 style='text-align: center; color: #2c3e50;'> 🐉 Bienvenido a Hong Kong 🐉</h1>", unsafe_allow_html=True)

    # Subtítulo con estilo
    st.markdown("<h2 style='text-align: center; color: #2980b9;'>Explora los Datos de Hong Kong</h2>", unsafe_allow_html=True)

    # Texto descriptivo con un poco de formato
    st.markdown("""
        <div style='text-align: center;'>
            <p style='font-size: 18px; color: #34495e;'>
                Hong Kong es una vibrante ciudad llena de cultura, historia y paisajes impresionantes. 
                Aquí hay algunas imágenes que muestran la belleza de la ciudad.
            </p>
        </div>
    """, unsafe_allow_html=True)

    # Mostrar imágenes de la ciudad con un estilo más atractivo
    image_list = [
        ("1.jpeg", "Skyline de Hong Kong"),
        ("2.jpeg", "Victoria Peak"),
        ("3.jpg", "Wong Tai Sin")
    ]

    # Crear columnas para mostrar las imágenes en un layout de 3 columnas
    cols = st.columns(3)  # Crea 3 columnas

    for i, (img_path, caption) in enumerate(image_list):
        with cols[i]:
            st.image(img_path, caption=f"Imagen {i+1}: {caption}", use_column_width=True)

# Regresion lineal simple
if Frames == "Regresion lineal simple":
    # Usar st.markdown para aplicar color
    st.markdown("<h1 style='text-align: center; color: darkblue;'>🌟 REGRESIÓN LINEAL SIMPLE</h1>", unsafe_allow_html=True)
    st.markdown("<h3 style='color: dodgerblue;'>Selecciona las variables:</h3>", unsafe_allow_html=True)

    # Agregar estilo a la barra lateral
    st.sidebar.markdown("<div style='background-color: #f0f2f6; padding: 10px; border-radius: 10px;'>", unsafe_allow_html=True)
    st.sidebar.markdown("**Opciones de visualización**", unsafe_allow_html=True)

    check_box = st.sidebar.checkbox(label="Mostrar Dataset")
    if check_box:
        st.write(Filtro_index1)
        st.write(df2)

    x_selected = st.sidebar.selectbox(label="🧮 x (Variable independiente)", options=numeric_cols)
    y_selected = st.sidebar.selectbox(label="📈 y (Variable dependiente)", options=numeric_cols)

    # Modelo y resultados
    X = numeric_df[x_selected].values.reshape(-1, 1)
    y = numeric_df[y_selected].values
    model = LinearRegression()
    model.fit(X, y)
    coef_Deter = model.score(X, y)
    coef_Correl = np.sqrt(coef_Deter)

    # Predecir los valores de y para la línea de tendencia
    y_pred = model.predict(X)

    # Crear gráfico de dispersión con la línea de tendencia
    figure1 = px.scatter(data_frame=numeric_df, x=x_selected, y=y_selected,
                         title='📊 Regresión lineal', color_discrete_sequence=["#1f77b4"],
                         template="plotly_dark")
    
    # Agregar línea de tendencia
    figure1.add_scatter(x=numeric_df[x_selected], y=y_pred, mode='lines', name='Línea de tendencia', line=dict(color='red', width=2))

    # Mostrar gráfico
    st.plotly_chart(figure1)

    st.markdown(f"<p style='font-size: 18px; color: blue;'>🔍 <strong>Coeficiente de correlación (R):</strong> {coef_Correl:.4f}</p>", unsafe_allow_html=True)

# Regresión lineal múltiple
if Frames == "Regresion lineal multiple":
    st.markdown("<h1 style='text-align: center; color: darkblue;'>🌟 REGRESIÓN LINEAL MÚLTIPLE</h1>", unsafe_allow_html=True)
    st.markdown("<h3 style='color: dodgerblue;'>Selecciona las variables:</h3>", unsafe_allow_html=True)

    # Seleccionar la variable dependiente (y) y las independientes (x)
    var_dep = st.sidebar.selectbox(label="📈 Variable dependiente (y)", options=numeric_cols)  # Cambia numeric_cols por tus columnas
    vars_indep = st.sidebar.multiselect(label="🧮 Variables independientes (x)", options=numeric_cols)

    if vars_indep and var_dep:
        # Declaramos las variables dependientes e independientes para la regresión lineal
        Vars_Indep = df[vars_indep]
        Var_Dep = df[var_dep]

        # Definimos y ajustamos el modelo de regresión lineal
        model = LinearRegression()
        model.fit(X=Vars_Indep, y=Var_Dep)

        # Predecimos los valores usando el modelo ajustado
        y_pred = model.predict(X=Vars_Indep)

        # Insertamos las predicciones en el DataFrame
        df['Predicciones'] = y_pred

        # Visualizamos los gráficos comparativos
        for var in vars_indep:
            # Gráfico de dispersión de la variable independiente vs total real
            figure = px.scatter(df, x=var, y=var_dep, title=f'Relación entre {var} y {var_dep}',
                                labels={var: var, var_dep: 'Total'},
                                color_discrete_sequence=["blue"], opacity=0.5)

            # Agregar las predicciones como puntos
            figure.add_scatter(x=df[var], y=df['Predicciones'], mode='markers', name='Predicciones', 
                               marker=dict(color='red', size=12, symbol='diamond', line=dict(width=2, color='black')), opacity=0.8)

            # Mostrar el gráfico
            st.plotly_chart(figure)

        # Coeficiente de determinación (R²) y coeficiente de correlación
        coef_Deter = model.score(Vars_Indep, Var_Dep)
        coef_Correl = np.sqrt(coef_Deter)

        st.markdown(f"<p style='font-size: 18px; color: blue;'>🔍 <strong>Coeficiente de correlación (R):</strong> {coef_Correl:.4f}</p>", unsafe_allow_html=True)
        st.markdown(f"<p style='font-size: 18px; color: blue;'>🔍 <strong>Coeficiente de determinación (R²):</strong> {coef_Deter:.4f}</p>", unsafe_allow_html=True)
    else:
        st.warning("Por favor, selecciona al menos una variable independiente y una variable dependiente.")

     # Definición de las funciones no lineales
def funcion_cuadratica(x, a, b, c):
    return a * x**2 + b * x + c

def funcion_exponencial(x, a, b, c):
    return a * np.exp(b * x) + c

def funcion_senoidal(x, a, b):
    return a * np.sin(x) + b

def funcion_tangencial(x, a, b):
    return a * np.tan(x) + b

def funcion_valor_absoluto(x, a, b, c):
    return a * np.abs(x) + b * x + c

def funcion_cociente_polinomios(x, a, b, c):
    return (a * x**2 + b) / (c * x)

def funcion_logaritmica(x, a, b):
    return a * np.log(x) + b

def funcion_polinomial_inversa(x, a, b, c):
    return a / b * x**2 + c * x

# Mejorando la presentación para la regresión no lineal
if Frames == "Regresion no lineal":
    # Título principal
    st.markdown("<h1 style='text-align: center; color: darkgreen;'>🌟 REGRESIÓN NO LINEAL 🌟</h1>", unsafe_allow_html=True)
    st.markdown("<h3 style='color: #2E86C1;'>Selecciona el tipo de función para realizar el ajuste:</h3>", unsafe_allow_html=True)

    # Selección de variables
    x_selected3 = st.sidebar.selectbox(label="📊 Variable independiente (X)", options=numeric_cols)
    y_selected3 = st.sidebar.selectbox(label="📈 Variable dependiente (Y)", options=numeric_cols)

    # Selección de función de ajuste
    funcion_selected = st.sidebar.selectbox("Selecciona la función de ajuste",
        options=["Función cuadrática", "Función exponencial", "Función senoidal", 
                 "Función tangencial", "Función Valor absoluto", "Función cociente entre polinomios", 
                 "Función logaritmica", "Función polinomial inversa"])

    # Diccionario de funciones
    funciones = {
        "Función cuadrática": funcion_cuadratica,
        "Función exponencial": funcion_exponencial,
        "Función senoidal": funcion_senoidal,
        "Función tangencial": funcion_tangencial,
        "Función Valor absoluto": funcion_valor_absoluto,
        "Función cociente entre polinomios": funcion_cociente_polinomios,
        "Función logaritmica": funcion_logaritmica,
        "Función polinomial inversa": funcion_polinomial_inversa
    }

    # Obtener los datos seleccionados
    X3 = numeric_df[x_selected3].values
    y3 = numeric_df[y_selected3].values
    
    try:
        # Ajustar la función seleccionada a los datos
        popt, _ = curve_fit(funciones[funcion_selected], X3, y3)

        # Predecir los valores ajustados
        y_pred3 = funciones[funcion_selected](X3, *popt)

        # Gráfico de dispersión y la curva ajustada
        fig = px.scatter(x=X3, y=y3, title=f'Ajuste de {funcion_selected}',
                         labels={'x': x_selected3, 'y': y_selected3},
                         template='plotly_white')

        # Agregar la línea de ajuste
        fig.add_scatter(x=X3, y=y_pred3, mode='lines', name=f'Ajuste {funcion_selected}', line=dict(color='red', width=2))

        # Mejoras visuales del gráfico
        fig.update_layout(
            title_font=dict(size=24, color='darkblue', family="Arial"),
            xaxis_title=dict(font=dict(size=16, color='darkblue')),
            yaxis_title=dict(font=dict(size=16, color='darkblue')),
            legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1)
        )

        # Mostrar el gráfico en Streamlit
        st.plotly_chart(fig)

        # Calcular el coeficiente de correlación
        correlacion3 = np.corrcoef(y3, y_pred3)[0, 1]

        # Mostrar el coeficiente de correlación con formato
        st.markdown(f"<p style='font-size: 18px; color: darkblue;'>🔍 <strong>Coeficiente de correlación (R):</strong> {correlacion3:.4f}</p>", unsafe_allow_html=True)

    except Exception as e:
        # Mensaje de error mejorado
        st.error(f"⚠️ Error al ajustar la función: {str(e)}")
        st.markdown("<p style='color: orange;'>⚠️ Verifica que la variable seleccionada sea adecuada para la función seleccionada (por ejemplo, el logaritmo solo admite valores positivos).</p>", unsafe_allow_html=True)

# HEATMAP
if Frames == "Heatmap":
    # Títulos y subtítulos más llamativos
    st.markdown("<h1 style='text-align: center; color: #2c3e50;'>Mapa de Calor</h1>", unsafe_allow_html=True)
    st.markdown("<h2 style='text-align: center; color: #2980b9;'>Correlaciones entre variables</h2>", unsafe_allow_html=True)
    
    # Crear la figura y el eje
    fig, ax = plt.subplots(figsize=(12, 10))  # Tamaño ajustado para mejor visualización

    # Generar el heatmap con ajustes estéticos
    sns.heatmap(
        numeric_df.corr(),
        annot=True,
        fmt=".2f",  # Formato de los números
        cmap="coolwarm",  # Mapa de colores
        ax=ax,
        cbar_kws={"shrink": .8},  # Ajusta el tamaño de la barra de color
        linewidths=0.5,  # Añade líneas de separación entre los cuadros
        linecolor='gray',  # Color de las líneas de separación
        square=True  # Hace que las celdas sean cuadradas
    )
    
    # Título del heatmap con formato mejorado
    ax.set_title("Heatmap de la Correlación", fontsize=18, color="#34495e", pad=20)
    
    # Ajustar los ticks y su estilo
    plt.xticks(fontsize=12, rotation=45, ha='right')  # Rota las etiquetas del eje x
    plt.yticks(fontsize=12)  # Ajusta el tamaño de las etiquetas del eje y
    
    # Mostrar el heatmap en Streamlit
    st.pyplot(fig)





Overwriting app.py
